In [1]:
import pytesseract
import openai
import re
import numpy as np
import os
from PIL import Image
from django.shortcuts import render
from django.http import JsonResponse

openai.api_key = os.getenv('#open ai 키입력') # open ai 키 입력

In [ ]:
def Text_From_Image(image): # 이미지에서 텍스트 추출
    try:
        extracted_text = pytesseract.image_to_string(image)
        return extracted_text.strip()
    except Exception as e:
        print(f"Error extracting text from image: {e}")
        return None

def Medication_Name(text): # 추출된 텍스트에서 약 이름 추출
    match = re.search(r'\b([A-Z][a-z]+)\b', text)
    if match:
        return match.group(1)
    else:
        return None

def Medication_Ask(medication_name): # chatgpt에 질문 전송하여 답변 받음
    try:
        question = f"무슨 약이야 {medication_name}?"
        response = openai.Completion.create(
            engine="text-davinci-003",  # ChatGPT 3.5
            prompt=question,
            max_tokens=150
        )
        return response.choices[0].text.strip()
    except Exception as e:
        print(f"Error asking GPT: {e}")
        return None

def index(request):
    return render(request, 'index.html')


def Process_Image(request): # 이미지 처리 및 약 정보 추출
    if request.method == 'POST' and request.FILES['image']: 
        image = request.FILES['image']
        pil_img = Image.open(image)
        
        extracted_text = Text_From_Image(pil_img)  # 이미지에서 텍스트 추출

        if extracted_text:
            medication_name = Medication_Name(extracted_text)  # 추출된 텍스트에서 약 이름 추출

            if medication_name:
                answer = Medication_Ask(medication_name)  # 생성형 AI에 약 이름에 대한 질문 전송 및 답변
                if answer:
                    return JsonResponse({'medication_name': medication_name, 'answer': answer})
                else:
                    return JsonResponse({'error': 'Failed to get a response from ChatGPT'})
            else:
                return JsonResponse({'error': 'No medication name found in the text'})
        else:
            return JsonResponse({'error': 'Failed to extract text from the image'})

    return JsonResponse({'error': 'Invalid request'})